In [1]:
# bigquery_helper.py
import streamlit as st
from google.cloud import bigquery
from google.oauth2 import service_account
import streamlit as st
import os
import numpy as np
import plotly.express as px
import pandas as pd
from streamlit_keplergl import keplergl_static
from keplergl import KeplerGl

CREDENTIALS_PATH = os.getenv('CREDENTIALS_PATH')
DATASET_NAME = os.getenv('DATASET_NAME')
TABLE_NAME = os.getenv('TABLE_NAME')


In [2]:
# Function to create BigQuery client
def create_bigquery_client(credentials_path):
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
    )
    client = bigquery.Client(credentials=credentials, location="europe-west9")
    return client

def query_all_from_table(credentials_path, dataset_name, table_name, query, location="US"):
    if len(query) < 1:
        query = f"SELECT * FROM `{dataset_name}.{table_name}` WHERE Total_duration__ms_ < 8000000 "
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
    )
    client = bigquery.Client(credentials=credentials, location=location)
    query_job = client.query(query)
    return query_job.result().to_dataframe()

### DEFINING HELPER FUNCTIONS

def clean_column_names(df):
    # Strip whitespace, lowercase, and replace spaces with underscores for each column name
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

def extract_neighborhood(df, column_name, neighborhood_column_name):
    # Split the column on the comma and take the second part (strip to remove leading/trailing spaces)
    df[neighborhood_column_name] = df[column_name].str.split(',').str[1].str.strip()
    return df


In [7]:
### QUERYING DATA

### FACT DATA

query = f"SELECT * FROM `{DATASET_NAME}.{TABLE_NAME}` WHERE Total_duration < 8000000 LIMIT 10000"
table_data = query_all_from_table(CREDENTIALS_PATH, dataset_name=DATASET_NAME, table_name=TABLE_NAME, location="US", query=query)

### GEOGRAPHY DATA

import requests

url = "https://api.tfl.gov.uk/BikePoint/"
req = requests.get(url)
df_geo = pd.DataFrame(req.json())
df_geo["station_id"] = df_geo.apply(lambda x: x["additionalProperties"][0]["value"], axis=1)
df_geo =  extract_neighborhood(df_geo, column_name = "commonName", neighborhood_column_name="nbh")

nbh_df = df_geo.groupby(by=["nbh"],as_index=False).agg(
        nbh_lat=('lat', 'mean'),
        nbh_lon=('lon', 'mean')
    )

### DATA PREP - JOINING

df_final = table_data.copy()
df_final = clean_column_names(df_final)
df_final =  extract_neighborhood(df_final, column_name = "start_station", neighborhood_column_name="start_nbh")
df_final = extract_neighborhood(df_final, column_name = "end_station", neighborhood_column_name="end_nbh")

df_final = df_final.merge(nbh_df, how="left", left_on ="start_nbh", right_on="nbh")
df_final.rename(mapper={"nbh_lat":"start_lat", "nbh_lon":"start_lon"},axis=1, inplace=1)

df_final = df_final.merge(nbh_df, how="left", left_on ="end_nbh", right_on="nbh")
df_final.rename(mapper={"nbh_lat":"end_lat", "nbh_lon":"end_lon"},axis=1, inplace=1)

df_final.drop(columns=["nbh_x", "nbh_y"],axis=1,inplace=True)
df_final.head()

### DATA PREP - AGGREGATING

df_agg = df_final.groupby(by=["start_nbh", "end_nbh", "start_lat","start_lon", "end_lat", "end_lon"],as_index=False).agg(
        count=('bike_number', 'count')
    )


top_five_df = df_agg.groupby("start_nbh").apply(
      lambda x: x.nlargest(1, "count")
  ).reset_index(drop=True)


In [8]:
map_1 = KeplerGl(height=700)
map_1.add_data(data=top_five_df, name='data_1')
map_1


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [10]:
import json
def save_dict_as_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

save_dict_as_json(map_1.config, "")


IsADirectoryError: [Errno 21] Is a directory: '/.'